In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex 客户端库：用于在线预测的 AutoML 表格回归模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_regression_online_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_regression_online_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建表格回归模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行在线预测。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。您将使用的数据集版本仅包括年份、月份和日期字段，用于预测平均每日温度（mean_temp）的值。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML表格回归模型，并部署用于在线预测。您也可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

费用

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装最新版本的 Vertex 客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *云存储*，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

开始之前

GPU运行时

*如果您有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得$300的免费信用额，可用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk)已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**提示**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改用于其余笔记本操作的“REGION”变量。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不能使用多区域存储桶对Vertex进行培训。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，可能会使用共享的测试帐户或项目。为避免资源创建时用户名称冲突，您可以为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，您的环境已经得到验证。请跳过此步骤。

**如果您正在使用Colab**，请运行下面的代码块，并按照提示进行身份验证以通过oAuth认证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐户授予项目访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，然后选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，然后选择**Storage Object管理权限**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的代码块中，将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该代码块。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库和定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点API服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常量

设置专属于AutoML数据集和训练的常量：

- 数据集模式：告诉“数据集”资源服务它是哪种类型的数据集。
- 数据标签（注释）模式：告诉“数据集”资源服务数据是如何标记（注释）的。
- 数据集训练模式：告诉“管道”资源服务为哪种任务（例如分类）训练模型。

In [ ]:
# Tabular Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
LABEL_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)
# Tabular Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

硬件加速器

设置硬件加速器（例如，GPU）用于预测。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，并为每个VM分配4个Nvidia Telsa K80 GPU，您应该指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

Docker容器映像

对于AutoML批量预测，由Vertex预测服务预先确定用于服务二进制的容器映像。更具体地，该服务将根据您选择的硬件加速器选择适当的容器用于模型。

#### 机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
  - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
  - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您还可以使用n2和e2机器类型进行培训和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的AutoML表格回归模型。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本）中，您将创建一个客户端，该客户端向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。所以提前设置好它们。

- 用于`Dataset`资源的数据集服务。
- 用于`Model`资源的模型服务。
- 用于训练的管道服务。
- 用于部署的终端点服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，您的第一步是创建一个`Dataset`资源实例。这一步与Vision、Video和Language不同。对于这些产品，在创建`Dataset`资源之后，然后可以使用`import_data`方法分别导入数据。

对于表格数据，数据的导入会推迟到训练流水线开始训练模型时。我们该怎么做呢？首先，您不会调用`import_data`方法。相反，当您创建数据集实例时，您将在`Dataset`资源的元数据中指定包含您表格数据的CSV文件或BigQuery位置的Cloud Storage位置。

#### Cloud Storage

`metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}`

Cloud Storage路径的格式为：

    gs://[bucket_name]/[folder(s)/[file]

#### BigQuery

`metadata = {"input_config": {"bigquery_source": {"uri": [gcs_uri]}}}`

BigQuery路径的格式为：

    bq://[collection].[dataset].[table]

请注意，`uri`字段是一个列表，您可以对多个CSV文件或BigQuery表进行输入，当您的数据分布在多个文件中时。

数据准备

对于表格数据，顶点`Dataset`资源具有一些要求。

- 必须是CSV文件或BigQuery查询。

#### BigQuery培训数据的位置。

现在将变量`IMPORT_FILE`设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"

#### 快速查看数据

您将使用存储在公共BigQuery表中的NOAA历史天气数据集的一个版本。

**TODO**
首先快速查看数据。通过计算表中行的数量 (`wc -l`) 来统计示例的数量，然后查看前几行数据。

In [ ]:
!bq head -n 10 $IMPORT_FILE

## 数据集

现在您的客户已经准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建一个 `Dataset` 资源实例。这个函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），包括以下参数：
- `display_name`：您选择的可读名称。
- `metadata_schema_uri`：数据集类型的模式。
- `metadata`：表格数据的云存储或BigQuery位置。
3. 调用客户端数据集服务的方法 `create_dataset`，包括以下参数：
- `parent`：您的 `Database`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用方式。虽然这个步骤通常很快，但在项目中首次使用时，由于配置的延迟，可能会有更长的延迟时间。

您可以使用 `operation` 对象来获取操作的状态（例如，创建 `Dataset` 资源）或取消操作，通过调用操作方法：

| 方法      | 描述 |
| ----------- | ----------- |
| result()    | 等待操作完成并以 JSON 格式返回结果对象。 |
| running()   | 返回操作是否仍在运行的 True/False。 |
| done()      | 返回操作是否已完成的 True/False。 |
| canceled()  | 返回操作是否已取消的 True/False。 |
| cancel()    | 取消操作（可能需要长达30秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, src_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        if src_uri.startswith("gs://"):
            metadata = {"input_config": {"gcs_source": {"uri": [src_uri]}}}
        elif src_uri.startswith("bq://"):
            metadata = {"input_config": {"bigquery_source": {"uri": [src_uri]}}}
        dataset = aip.Dataset(
            display_name=name,
            metadata_schema_uri=schema,
            labels=labels,
            metadata=json_format.ParseDict(metadata, Value()),
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("gsod-" + TIMESTAMP, DATA_SCHEMA, src_uri=IMPORT_FILE)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

## 训练模型

现在使用您的 Vertex `Dataset` 资源来训练一个 AutoML 表格回归模型。 要训练模型，请执行以下步骤：

1. 为 `Dataset` 资源创建一个 Vertex 训练管道。
2. 执行管道以开始训练。

### 创建一个训练流水线

您可能会问，我们用流水线做什么？通常在工作（如训练）具有多个步骤时，您会使用流水线，通常按顺序进行：执行步骤A，执行步骤B，依此类推。通过将这些步骤放入流水线中，我们可以获得以下好处：

1. 对后续训练作业具有重复使用性。
2. 可以容器化并作为批处理作业运行。
3. 可以进行分布式处理。
4. 所有步骤都与同一流水线作业相关联，以跟踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：流水线作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex的完全限定数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练作业要求的字典。

辅助函数调用 `Pipeline` 客户端服务方法 `create_pipeline`，它接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：流水线训练作业的完整规范。

现在让我们更深入地看一下构建 `training_pipeline` 规范所需的*最低*要求：

- `display_name`：流水线作业的人类可读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
   - `dataset_id`：只有 Vertex 数据集标识符（非完全限定）--这是完全限定标识符的最后部分。
   - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比会自动由 AutoML 选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与接受 Python（类似 JSON）字典的其他参数不同，`task`字段接受一个谷歌 protobuf 结构体，这与 Python 字典非常相似。使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段包括：

- `prediction_type`：我们是在进行"分类"还是"回归"。
- `target_column`：我们要预测的 CSV 表头列的名称（即标签）。
- `train_budget_milli_node_hours`：为训练模型预算的最长时间，其中 1000 = 1小时。
- `disable_early_stopping`：为了让 AutoML 自行决定是否提前停止训练或将整个预算用于训练而设置为 True/False。
- `transformations`：为每个特征列指定特征工程。

对于`transformations`，列表必须为每一列包含一个条目。外部关键字段指示相应列的特征工程类型。在本教程中，您将其设置为`"auto"`，以告诉 AutoML 自动确定它。

最后，通过调用辅助函数`create_pipeline`来创建管道，该函数将返回训练管道对象的实例。

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

In [ ]:
PIPE_NAME = "gsod_pipe-" + TIMESTAMP
MODEL_NAME = "gsod_model-" + TIMESTAMP

task = Value(
    struct_value=Struct(
        fields={
            "target_column": Value(string_value=label_column),
            "prediction_type": Value(string_value="regression"),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "disable_early_stopping": Value(bool_value=False),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关训练管道的信息

现在仅获取此训练管道实例的管道信息。通过调用作业客户服务的`get_training_pipeline`方法，helper function获取此作业的作业信息，参数如下：

- `name`：Vertex的完全限定管道标识符。

当模型训练完成时，管道状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练上述模型可能需要长达30分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的Vertex模型资源标识符，即管道服务分配给它的标识符。您可以从返回的管道实例中获取此信息，作为字段`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在你的模型已经训练好了，你可以获取一些关于你的模型的信息。

评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被设置为测试（留存）数据，这些数据被管道服务用来评估模型。

### 列出所有切片的评估

使用这个辅助函数 `list_model_evaluations`，它接受以下参数：

- `name`：`Model` 资源的 Vertex 完全限定的模型标识符。

这个辅助函数使用模型客户端服务的 `list_model_evaluations` 方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 你可能只有一个，然后我们打印出每个评估指标中的所有关键名称，并且对于一小组指标（`rootMeanSquaredError` 和 `meanAbsoluteError`）你将打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("rootMeanSquaredError", metrics["rootMeanSquaredError"])
        print("meanAbsoluteError", metrics["meanAbsoluteError"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

部署“Model”资源

现在部署使用AutoML创建的已训练的Vertex“Model”资源。这需要两个步骤：

1. 为将“Model”资源部署到创建的“Endpoint”资源。

2. 将“Model”资源部署到“Endpoint”资源。

### 创建 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个用于提供预测服务的端点，并传入以下参数：

- `display_name`: `Endpoint` 资源的一个可读名称。

这个辅助函数使用端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`: `Endpoint` 资源的一个可读名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为为了提供服务，可能需要一些时间来设置 `Endpoint` 资源。您可以调用 `response.result()`，这是一个同步调用，当 `Endpoint` 资源准备好时将返回。这个辅助函数返回 `Endpoint` 资源的 Vertex 完全合格标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "gsod_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“端点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

在处理您的在线预测请求时，您有几种扩展计算实例的选择：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例间分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，将部署固定数量的计算实例，并将在线预测请求均匀分布在它们之间。

- 自动扩展：在线预测请求在可扩展数量的计算实例间分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为在模型首次部署时要提供和取消提供的数量，并将最大（`MAX_NODES`）数量设置为根据负载情况要提供的计算实例数量。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 将`Model`资源部署到`Endpoint`资源

使用这个辅助函数`deploy_model`将`Model`资源部署到您为提供预测创建的`Endpoint`资源，具有以下参数：

- `model`: 要从训练管道上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读名称。
- `endpoint`: 要部署模型的 Vertex 完全限定终端标识符。

辅助函数调用`Endpoint`客户端服务的`deploy_model`方法，该方法接受以下参数：

- `endpoint`: 要将`Model`资源部署到的 Vertex 完全限定`Endpoint`资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 在终端中传输到该模型的流量百分比，指定为一个或多个键值对的字典。
   - 如果只有一个模型，则指定为**{ "0": 100 }**，其中"0"指的是要上传的模型，100表示100％的流量。
   - 如果在终端上有现有模型，要分配流量，则使用`model_id`指定为**{ "0": percent, model_id: percent, ... }**，其中`model_id`是要部署到的终端的现有模型的模型标识符。百分比必须加起来等于100。

现在让我们深入研究`deployed_model`参数。该参数指定为具有最小必填字段的Python字典：

- `model`: 要部署的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读名称。
- `disable_container_logging`: 禁用容器事件的记录，例如执行失败（默认情况下启用容器记录）。在调试部署时通常启用容器记录，部署用于生产时再禁用。
- `dedicated_resources`: 指定为为提供预测请求扩展多少计算实例（副本）。
  - `machine_spec`: 要提供的计算实例。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。
  - `min_replica_count`: 最初要提供的计算实例数，您之前设置为变量`MIN_NODES`。
  - `max_replica_count`: 要扩展到的计算实例的最大数量，您之前设置为变量`MAX_NODES`。

#### 流量分流

让我们现在深入研究`traffic_split`参数。该参数指定为Python字典。起初可能会有点混乱。让我解释一下，您可以将模型的多个实例部署到一个终端，然后设置每个实例接收多少（百分比）的流量。

为什么要这样做？也许您已经在生产环境中部署了以前的版本，我们称之为 v1。您对 v2 的模型评估更好，但在实际部署到生产环境之前，您并不确定它是否真的更好。因此，在流量分流的情况下，您可能希望将 v2 部署到与 v1 相同的终端，但只得到 10% 的流量。这样，您可以在不干扰大多数用户的情况下监视其表现，直到您做出最终决定。

#### 响应

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`来等待操作完成，该方法将阻塞直到模型被部署。如果这是首次将模型部署到终端，则可能需要额外几分钟来完成资源的分配。

In [ ]:
DEPLOYED_NAME = "gsod_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

进行在线预测请求，现在对部署的模型进行在线预测。

制作测试项目

您将使用合成数据作为测试数据项目。不要担心我们使用的是合成数据 - 我们只是想演示如何做出预测。

In [ ]:
INSTANCE = {"year": "1932", "month": "11", "day": "6"}

### 进行预测

现在你有了一个测试项目。使用这个辅助函数 `predict_item`，它接受以下参数：

- `filename`：测试项目的云存储路径。
- `endpoint`：`Model` 资源部署的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `parameters_dict`：用于提供预测结果的额外过滤参数。

此函数使用以下参数调用预测客户端服务的 `predict` 方法：

- `endpoint`：`Model` 资源部署的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `instances`：要预测的实例（数据项）列表。
- `parameters`：用于提供预测结果的额外过滤参数。*注意*，表格模型不支持额外参数。

#### 请求

每个实例的格式是，其中值必须指定为字符串：

    { 'feature_1': 'value_1', 'feature_2': 'value_2', ... }

由于 `predict()` 方法可以接受多个项目（实例），您将单个测试项目作为一个测试项目列表发送。最后一步，您将实例列表封装到Google的 protobuf 格式中，这是我们传递给 `predict()` 方法的格式。

#### 响应

`response` 对象返回一个列表，列表中的每个元素对应请求中的相应图像。您将在每个预测的输出中看到：
- `value`：预测的值。

In [ ]:
def predict_item(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [data]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))


predict_item(INSTANCE, endpoint_id, None)

## 取消部署 `Model` 资源

现在从服务`Endpoint`资源中取消部署您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：当`Model`资源被部署到时，终端服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的 Vertex 完全合格标识符。

这个函数调用端点客户服务的方法`undeploy_model`，使用以下参数：

- `deployed_model_id`：终端服务在部署`Model`资源时返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的 Vertex 完全合格标识符。
- `traffic_split`：如何在`Endpoint`资源上的其他部署模型之间分配流量。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将`traffic_split`设置为空对象{}来保持它为空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理该项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME